# Trabajo Final procesamiento de lenguaje natural (NLP) <a class="tocSkip">
## Universidad Pontificia Bolivariana <a class="tocSkip">

**Estudiante:** Ivan Andres Velasco Arias

**ID:** 000582575



**Subir Este notebook de Trabajo Final en su repositorio de GitHub. y enviar el link en **Microsoft TEAMS**

Docente: [Jose R. Zapata](https://joserzapata.github.io)
- https://joserzapata.github.io
- https://www.linkedin.com/in/jose-ricardo-zapata-gonzalez/       


## Objetivo del Trabajo
Realizar un proceso de Procesamiento de lenguaje natural (NLP) para dejar los datos preparados para ser usados con algoritmos de Machine Learning para Clasificación como objetivo final del trabajo.

los datos preparados para ser usados con algoritmos de Machine Learning para Regresión o Clasificación como objetivo final del trabajo.

El trabajo se realizara en este jupyter notebook y subirlo a su repositorio de github creado en clase. (**Recuerde poner su nombre e información**)

## Las actividades a realizar
    

1) Limpiar los datos de texto.

   - https://joserzapata.github.io/courses/nlp/procesamiento-basico/

   - https://joserzapata.github.io/courses/nlp/preprocesamiento-texto/

2) Realizar la representación de texto:

   - https://joserzapata.github.io/courses/nlp/representaciones/

   - Tokenización
   - Lematización o stemming
   - Representación de los datos de texto (Bag of Words o TF-IDF)

3) Utilice un modelo de Machine Learning para clasificación para entrenar y evaluar el modelo.
        
    - Calcule estas las métricas de evaluación (accuracy, precision, recall, f1-score) - https://joserzapata.github.io/courses/python-ciencia-datos/clasificacion/#evaluacion-modelo-simple


    
   
    
*NOTA: No dude en contactarme para cualquier pregunta o inquietud :) por el chat de Teams o al correo
joser.zapata@upb.edu.co*

## EVALUACIÓN


|Porcentaje en la evaluación | Descripción| Nada | Incompleto | Completo 
| :---: |:---: |:---: |:---: |:---: 
| 5 % |**limpieza de los datos** |
| 5 % | **Representacion de los datos** |
| 5 % | **Machine Learning** <br> Entrenar y evaluar el modelo propuesto |


In [63]:
import pandas as pd
import numpy as np

from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    classification_report,
)
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import nltk

# Descargar stopwords en español
nltk.download("stopwords")
from nltk.corpus import stopwords

# Obtener stopwords en español
stop_words = set(stopwords.words("spanish"))


[nltk_data] Downloading package stopwords to /home/vscode/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Carga del dataset

El archivo  se carga en aproximadamente 10 segundos

In [45]:
# Usar su ID UPB Ejemplo: "0028984798"
id_upb = "000582575"
 
data_reviews = pd.read_parquet(
    "https://www.dropbox.com/scl/fi/gvk9yj8cn96oocr9z058x/filmaffinity_reviews_dataset.parquet?rlkey=xgvr00zvkxbkwqqavqutpsshg&st=xjb7xze9&dl=1",
    engine="fastparquet"
)
data_reviews = data_reviews.sample(n=50_000, random_state=int(id_upb))
data_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50000 entries, 127841 to 68335
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   author_review_desc  50000 non-null  object
 1   author_rating       50000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.1+ MB


Ejemplo de algunas filas del dataset

In [46]:
data_reviews.sample(5)

,author_review_desc,author_rating
24077,\nEs sintomático que esta película haya tardad...,7
4471,\nPor lo general Zack Snyder es un director qu...,2
76877,\nMe lo he pasado teta viendo esta película. E...,8
91938,\nUno va al cine a ver esta película con ciert...,5
106279,\nPelícula sorprendente que trata el tema de l...,7


Evaluar los valores nulos

In [47]:
data_reviews.isnull().sum()

author_review_desc    0
author_rating         0
dtype: int64

En este caso no hay nulos.

Revisar si hay valores duplicados:

In [48]:
data_reviews.duplicated().sum()

np.int64(2)

No hay  valores duplicados.

## Clasificación Tradicional para Análisis de Sentimientos y Categorías 👍 👎



Definición: Se considera reseña positiva cuando la puntuación ("author_rating") es mayor que 6; negativa en caso contrario.

Crear la variable binaria de sentimiento: 1 (positivo) si author_rating > 6, 0 (negativo) de lo contrario

In [49]:
UMBRAL = 6
data_reviews["sentiment_bin"] = (data_reviews["author_rating"] > UMBRAL).astype(int)

In [50]:
# Dividir los datos en conjuntos de entrenamiento y prueba

X_data = data_reviews["author_review_desc"]
y_data = data_reviews["sentiment_bin"]


Dividir el dataset en entrenamiento y prueba

In [51]:
X_train, X_test, y_train, y_test = train_test_split(
    X_data,
    y_data,
    test_size=0.2,
    stratify=y_data,  # Mantener la proporción de clases en ambos conjuntos
    random_state=42,
)

## Análisis exploratorio de los datos

para determinar que tipo de limpieza se debe realizar a los datos de texto de `X_train`

In [52]:
X_train


19328     \nCuando se habla de Everest, hay que distingu...
110861    \nLe doy un 6.\n\nLe doy un 9 bajo los efectos...
69646     \nYa en los créditos iniciales Andrew Dominik ...
94461     \nFue la primera película que vi, en la que se...
34522     \nPeliculón grandioso para la historia del cin...
                                ...                        
136346    \nLa serie te engancha con un arco argumental ...
10238     \nFúsi es un cuarentón, soltero, gigante de es...
74013     \nNo he leído a nadie que se queje de la cámar...
121614    \nVuelve Jason Reitman a hacer una película so...
38682     \nCon ocasión de la cena de navidad de 1907, e...
Name: author_review_desc, Length: 40000, dtype: object

In [53]:
X_train.shape


(40000,)

In [54]:
X_train

19328     \nCuando se habla de Everest, hay que distingu...
110861    \nLe doy un 6.\n\nLe doy un 9 bajo los efectos...
69646     \nYa en los créditos iniciales Andrew Dominik ...
94461     \nFue la primera película que vi, en la que se...
34522     \nPeliculón grandioso para la historia del cin...
                                ...                        
136346    \nLa serie te engancha con un arco argumental ...
10238     \nFúsi es un cuarentón, soltero, gigante de es...
74013     \nNo he leído a nadie que se queje de la cámar...
121614    \nVuelve Jason Reitman a hacer una película so...
38682     \nCon ocasión de la cena de navidad de 1907, e...
Name: author_review_desc, Length: 40000, dtype: object

In [55]:
# ...existing code...
# Análisis exploratorio simple para X_train
import re
from collections import Counter
import numpy as np

texts = X_train.dropna().astype(str).reset_index(drop=True)

# Estadísticas básicas
n = len(texts)
lengths = texts.str.len()
tokens_per = texts.str.split().map(len)

print("Filas:", n)
print(
    "Longitud chars - min / med / max:",
    int(lengths.min()),
    int(lengths.median()),
    int(lengths.max()),
)
print(
    "Tokens por reseña - min / med / max:",
    int(tokens_per.min()),
    int(tokens_per.median()),
    int(tokens_per.max()),
)
print("Pct <5 tokens: {:.1%}".format((tokens_per < 5).mean()))
print("Pct >200 chars: {:.1%}".format((lengths > 200).mean()))

# Detectar elementos especiales
re_url = re.compile(r"https?://\S+|www\.\S+")
re_num = re.compile(r"\d+")
re_html = re.compile(r"<[^>]+>")
re_emoji = re.compile(
    "[\U0001f600-\U0001f64f\U0001f300-\U0001f5ff\U0001f680-\U0001f6ff\U0001f1e0-\U0001f1ff]+",
    flags=re.UNICODE,
)


def frac(pat):
    return texts.str.contains(pat).mean()


print("\nFracciones especiales:")
print("URLs: {:.2%}".format(frac(re_url)))
print("Números: {:.2%}".format(frac(re_num)))
print("HTML tags: {:.2%}".format(frac(re_html)))
print("Emojis/símbolos: {:.2%}".format(frac(re_emoji)))
print(
    "No-ASCII (posible caracteres especiales): {:.2%}".format(
        texts.apply(lambda s: s.encode("ascii", errors="ignore").decode() != s).mean()
    )
)

# Top palabras (tokenización sencilla, sólo letras)
word_re = re.compile(r"\b[a-záéíóúñü]+\b", flags=re.IGNORECASE)
counter = Counter()
for s in texts.str.lower():
    counter.update(word_re.findall(s))
top20 = counter.most_common(20)
print("\nTop 20 palabras (simple):")
for w, c in top20:
    print(f"{w} : {c}")

# Ejemplos útiles
print("\nEjemplos (primeras 3 reseñas):")
for e in texts.head(3):
    print("-", e[:200].replace("\n", " "))

print("\nEjemplos con URL / emoji / muy cortas:")
print("Con URL (hasta 3):")
print(texts[texts.str.contains(re_url)].head(3).tolist())
print("Con emoji (hasta 3):")
print(texts[texts.str.contains(re_emoji)].head(3).tolist())
print("Muy cortas (<10 chars):", texts[texts.str.len() < 10].head(5).tolist())
# ...existing code...


Filas: 40000
Longitud chars - min / med / max: 28 1082 6031
Tokens por reseña - min / med / max: 3 185 1107
Pct <5 tokens: 0.0%
Pct >200 chars: 98.8%

Fracciones especiales:
URLs: 3.94%
Números: 43.47%
HTML tags: 0.32%
Emojis/símbolos: 0.00%
No-ASCII (posible caracteres especiales): 99.72%

Top 20 palabras (simple):
de : 514890
que : 380823
la : 343628
y : 292404
en : 229484
el : 216412
a : 188712
un : 143320
es : 137519
una : 132536
no : 126394
los : 115717
con : 107823
se : 105296
por : 90445
lo : 85707
del : 74457
su : 73650
las : 69364
película : 66947

Ejemplos (primeras 3 reseñas):
-  Cuando se habla de Everest, hay que distinguir entre su calidad como película, como producto cinematográfico, y Everest como crónica emotiva de uno de los más trágicos desastres acaecidos en la monta
-  Le doy un 6.  Le doy un 9 bajo los efectos de las drogas.   Humor, acción, tias buenas y superpoderes. Muchos no necesitan más que esto para disfrutar de una película. 
-  Ya en los créditos iniciale

## 1) Limpieza de los datos de texto

Tomar los datos de `X_train` y aplicar las funciones de limpieza que considere necesarias

Ayudas:

- Convertir a minúsculas
- Eliminar caracteres especiales y números
- Corregir palabras mal escritas
- etc


In [56]:
# Celda: Limpieza simple de texto
import re
import pandas as pd


def clean_text(s: str) -> str:
    if pd.isna(s):
        return ""
    s = str(s).lower()
    # Quitar URLs, emails y tags HTML
    s = re.sub(r"https?://\S+|www\.\S+", " ", s)
    s = re.sub(r"\S+@\S+\.\S+", " ", s)
    s = re.sub(r"<[^>]+>", " ", s)
    # Quitar números
    s = re.sub(r"\d+", " ", s)
    # Conservar letras (incluye vocales acentuadas, ñ, ü) y espacios; eliminar otros símbolos
    s = re.sub(r"[^a-záéíóúñü\s]", " ", s)
    # Normalizar espacios
    s = re.sub(r"\s+", " ", s).strip()
    # Tokenización y eliminación de stopwords
    tokens = s.split()
    tokens = [token for token in tokens if token not in stop_words]

    # Unir tokens sin stopwords
    s = " ".join(tokens)
    return s


# Aplicar limpieza a X_train y crear X_train_clean
X_train_clean = X_train.fillna("").astype(str).map(clean_text)

# Mostrar resultados resumidos
print("Ejemplos (primeras 5) — texto original -> texto limpio:")
for orig, clean in zip(X_train.head(5).tolist(), X_train_clean.head(5).tolist()):
    print("----")
    print("ORIG:", (orig or "")[:200].replace("\n", " "))
    print("CLEAN:", clean[:200])

# Aplicar la limpieza a todo X_train
print("Limpiando X_train...")
X_train_limpio = X_train.apply(clean_text)

# Aplicar la limpieza a X_test para mantener consistencia
print("Limpiando X_test...")
X_test_limpio = X_test.apply(clean_text)

# Verificar las dimensiones
print("\nDimensiones de los datos:")
print(f"X_train_limpio: {X_train_limpio.shape}")
print(f"X_test_limpio: {X_test_limpio.shape}")


Ejemplos (primeras 5) — texto original -> texto limpio:
----
ORIG:  Cuando se habla de Everest, hay que distinguir entre su calidad como película, como producto cinematográfico, y Everest como crónica emotiva de uno de los más trágicos desastres acaecidos en la monta
CLEAN: habla everest distinguir calidad película producto cinematográfico everest crónica emotiva trágicos desastres acaecidos montaña alta mundo primero triunfa segundo fracasa resultado decir película nota
----
ORIG:  Le doy un 6.  Le doy un 9 bajo los efectos de las drogas.   Humor, acción, tias buenas y superpoderes. Muchos no necesitan más que esto para disfrutar de una película. 
CLEAN: doy doy bajo efectos drogas humor acción tias buenas superpoderes necesitan disfrutar película
----
ORIG:  Ya en los créditos iniciales Andrew Dominik deja claro que no se trata de la típica producción hollywoodiense, más bien parece una presentación de una película de los setenta de autor muy “indi”, com
CLEAN: créditos iniciales and

## 2) Representacion del texto

Luego de tener los datos limpios, realizar la representación de los datos de texto para poder usarse en modelos de machine learning.

In [59]:
cv = CountVectorizer()
bow_matrix = cv.fit_transform(X_train_limpio)

print("Dimensiones de la matriz Bag-of-Words:", bow_matrix.shape)
print(
    "Ejemplo de términos (BoW):",
    np.random.choice(cv.get_feature_names_out(), size=10, replace=False),
)


Dimensiones de la matriz Bag-of-Words: (40000, 140254)
Ejemplo de términos (BoW): ['idealizaría' 'corales' 'hiperfantasma' 'cambiaran' 'mitic' 'calurosos'
 'vellocino' 'eviscerado' 'supervalorada' 'calvinismo']


In [60]:
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(X_train_limpio)

print("Dimensiones de la matriz TF-IDF:", tfidf_matrix.shape)
print(
    "Ejemplo de términos (TF-IDF):",
    np.random.choice(tfidf.get_feature_names_out(), size=10, replace=False),
)


Dimensiones de la matriz TF-IDF: (40000, 140254)
Ejemplo de términos (TF-IDF): ['alargo' 'esquematización' 'desmenuzo' 'devoluciones' 'bicicleta' 'meets'
 'presentarnoslo' 'docencia' 'relinchos' 'deméritos']


In [61]:
# Obtener nombres de términos y TF-IDF promedio por término en todo el corpus
feature_names = tfidf.get_feature_names_out()
mean_tfidf = np.asarray(tfidf_matrix.mean(axis=0)).ravel()

# Normalizar para interpretar como "probabilidades" (suma 1)
total = mean_tfidf.sum()
probs = mean_tfidf / total if total > 0 else np.zeros_like(mean_tfidf)

# DataFrame con término, tfidf medio y probabilidad normalizada
df_terms = pd.DataFrame(
    {
        "term": feature_names,
        "mean_tfidf": mean_tfidf,
        "probability": probs,
    }
)

print(
    "\nEjemplo aleatorio de 10 términos con su probabilidad (TF-IDF medio normalizado):"
)
print(
    df_terms.sample(10, random_state=2025)
    .sort_values("probability", ascending=False)
    .reset_index(drop=True)
)



Ejemplo aleatorio de 10 términos con su probabilidad (TF-IDF medio normalizado):
              term  mean_tfidf   probability
0             rock    0.000708  8.207116e-05
1        ninotchka    0.000015  1.779570e-06
2           miklós    0.000009  9.944111e-07
3         fraseado    0.000006  7.420270e-07
4           bullas    0.000005  5.266733e-07
5             oide    0.000004  4.875247e-07
6        entrecasa    0.000004  4.854708e-07
7           viñedo    0.000004  4.152911e-07
8  condicionándola    0.000003  3.920247e-07
9        profanada    0.000003  3.006788e-07


## 3) Entrenar un modelo de machine learning de clasificación

Utilizar un modelo de clasificación para entrenar y evaluar el modelo con los datos preparados.


In [64]:
lr_classifier = LogisticRegression()
lr_classifier.fit(tfidf_matrix, y_train)


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


### Evaluar el modelo con los datos de prueba

Usar el modelo para predecir en `X_test` y evaluar con `y_test`

**Nota:** Recuerde que `X_test` debe pasar por los mismos procesos de limpieza y representación que `X_train`

In [68]:
# Usar el transformador TF-IDF entrenado  con lo datos de train
# para procesar los datos de prueba
X_test_processed = tfidf.transform(X_test_limpio)

y_pred = lr_classifier.predict(X_test_processed)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Reporte de clasificación:\n", classification_report(y_test, y_pred))


Accuracy: 0.8091
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.81      0.77      0.79      4644
           1       0.81      0.84      0.83      5356

    accuracy                           0.81     10000
   macro avg       0.81      0.81      0.81     10000
weighted avg       0.81      0.81      0.81     10000



In [69]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))
print("ROC AUC Score:", roc_auc_score(y_test, y_pred))
print("Reporte de clasificación:\n", classification_report(y_test, y_pred))

Accuracy: 0.8091
Precision: 0.8076030697840443
Recall: 0.8448469006721434
F1 Score: 0.8258052742038507
ROC AUC Score: 0.8063597121793104
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.81      0.77      0.79      4644
           1       0.81      0.84      0.83      5356

    accuracy                           0.81     10000
   macro avg       0.81      0.81      0.81     10000
weighted avg       0.81      0.81      0.81     10000



**Guardar el modelo**

In [73]:
import joblib

joblib.dump(lr_classifier, "models/lr_IAVA_model.pkl")


['models/lr_IAVA_model.pkl']

## Librerías Usadas

In [74]:
from watermark import watermark

print(watermark(python=True, iversions=True, globals_=globals()))

Python implementation: CPython
Python version       : 3.12.11
IPython version      : 9.6.0

nltk     : 3.9.2
re       : 2.2.1
sklearn  : 1.7.2
sys      : 3.12.11 (main, Jul  1 2025, 05:29:09) [GCC 12.2.0]
watermark: 2.5.0
numpy    : 2.3.4
pandas   : 2.3.3
joblib   : 1.5.2



---

## Referencias
- [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)
- [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)
- [Classification of text documents using sparse features](https://scikit-learn.org/stable/auto_examples/text/plot_document_classification_20newsgroups.html)
- Ejemplo de Entrenamiento y selección de Modelo de machine learning entre varios modelos <https://joserzapata.github.io/post/ciencia-datos-proyecto-python/6-model_selection/>
- https://joserzapata.github.io/courses/python-ciencia-datos/python/
- https://joserzapata.github.io/courses/python-ciencia-datos/pandas/
- https://joserzapata.github.io/courses/python-ciencia-datos/machine-learning/

Docente: [Jose R. Zapata](https://joserzapata.github.io)

- [https://joserzapata.github.io/](https://joserzapata.github.io/)
- [https://www.linkedin.com/in/jose-ricardo-zapata-gonzalez/](https://www.linkedin.com/in/jose-ricardo-zapata-gonzalez/)